# Rossmann Sales Prediction
The goal of the project is to predict sales from different features of the dataset and find trends and patterns.

##  Import Libraries

In [ ]:
# Import libraries and mount drive
import numpy as np
import math as math
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import sklearn as sks
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, accuracy_score, precision_score, recall_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Data Cleaning and Light Data Exploration

In [ ]:
# Load the dataframe
store = pd.read_csv('/content/drive/MyDrive/Mynt Project/store.csv')
store

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1110,1111,a,a,1900.0,6.0,2014.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct"
1111,1112,c,c,1880.0,4.0,2006.0,0,NaN,NaN,NaN
1112,1113,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN
1113,1114,a,c,870.0,NaN,NaN,0,NaN,NaN,NaN


### Light Data Exploration for Store Dataframe

In [ ]:
# Check the information of the dataframes
store.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Store                      1115 non-null   int64  
 1   StoreType                  1115 non-null   object 
 2   Assortment                 1115 non-null   object 
 3   CompetitionDistance        1112 non-null   float64
 4   CompetitionOpenSinceMonth  761 non-null    float64
 5   CompetitionOpenSinceYear   761 non-null    float64
 6   Promo2                     1115 non-null   int64  
 7   Promo2SinceWeek            571 non-null    float64
 8   Promo2SinceYear            571 non-null    float64
 9   PromoInterval              571 non-null    object 
dtypes: float64(5), int64(2), object(3)
memory usage: 87.2+ KB


Assessment:
1. Determine reasons and patterns for missing values in the CompetitionOpenSinceMonth and CompetitionOpenSinceYear
2. There are huge portions of missing values for columns Promo2, Promo2SinceWeek, Promo2SinceYear, PromoInterval
3. Find patterns and reasons for missing values for the rest of the columns

In [ ]:
# Delete the rows with missing values for CompetitionDistance Column
store['CompetitionDistance'].dropna(inplace=True)

In [ ]:
# Check for the information again
store.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Store                      1115 non-null   int64  
 1   StoreType                  1115 non-null   object 
 2   Assortment                 1115 non-null   object 
 3   CompetitionDistance        1112 non-null   float64
 4   CompetitionOpenSinceMonth  761 non-null    float64
 5   CompetitionOpenSinceYear   761 non-null    float64
 6   Promo2                     1115 non-null   int64  
 7   Promo2SinceWeek            571 non-null    float64
 8   Promo2SinceYear            571 non-null    float64
 9   PromoInterval              571 non-null    object 
dtypes: float64(5), int64(2), object(3)
memory usage: 87.2+ KB


In [ ]:
# Check the value counts of store and train dataframes
for column in store.columns:
  print(store[column].value_counts())

In [ ]:
# Check the rows with missing values
store[store.isnull().any(axis=1)]

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN
5,6,a,a,310.0,12.0,2013.0,0,NaN,NaN,NaN
6,7,a,c,24000.0,4.0,2013.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1109,1110,c,c,900.0,9.0,2010.0,0,NaN,NaN,NaN
1111,1112,c,c,1880.0,4.0,2006.0,0,NaN,NaN,NaN
1112,1113,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN
1113,1114,a,c,870.0,NaN,NaN,0,NaN,NaN,NaN


Stores which are not participating in the promo have missing values for Promo2SinceWeek/Year and Promo Interval

Store with columns CompetitionOpenSinceMonth and CompetitionOpenSinceYear having missing values might not be having any competitions nearby

Since this is a supplemental data, we don't need to eliminate any values from the dataframe, instead we would fill in any missing values and create store clusters using KMeans Algorithm.

#### Feature Engineering

In [ ]:
# Fill missing values
store[['CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear']].fillna(0, inplace=True)
store['PromoInterval'].fillna('None')

# Create another column for Competition
Has_Competition = list()
for value in store['CompetitionOpenSinceMonth']:
  if value == 0:
    Has_Competition.append('No')
  else:
    Has_Competition.append('Yes')

store['HasCompetition'] = pd.DataFrame(Has_Competition)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [ ]:
# Create another column for Promo
Has_Promo = list()
for item in store['Promo2SinceWeek']:
  if item == 0:
    Has_Promo.append('No')
  else:
    Has_Promo.append('Yes')

store['HasPromo'] = pd.DataFrame(Has_Promo)
store

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,HasCompetition,HasPromo
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,Yes,Yes
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",Yes,Yes
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",Yes,Yes
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN,Yes,Yes
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN,Yes,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...
1110,1111,a,a,1900.0,6.0,2014.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct",Yes,Yes
1111,1112,c,c,1880.0,4.0,2006.0,0,NaN,NaN,NaN,Yes,Yes
1112,1113,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN,Yes,Yes
1113,1114,a,c,870.0,NaN,NaN,0,NaN,NaN,NaN,Yes,Yes


#### Clustering Algorithm

##### Data Preprocessing

Consider using OrdinalEncoder here

In [ ]:
# Separate the numerical and categorical variables
store_cat = store[['StoreType', 'Assortment', 'PromoInterval', 'HasCompetition', 'HasPromo']]
store_num = store.drop(columns=['StoreType', 'Assortment', 'PromoInterval', 'HasCompetition', 'HasPromo'])

In [ ]:
# Instiate the OneHot Encoder
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')

# Fit the categorical values to the ohe
ohe_store = ohe.fit_transform(store_cat)

# Instantiate the Standard Scaler
scaler = StandardScaler()

# Fit the scaler
store = scaler.fit_transform(store_num)


# Import the model
from sklearn.cluster import KMeans

# Instantiate the model
kmeans = KMeans()

# Make a pipeline
store_pipe = Pipeline(
    steps=(['OneHot Encoder', ohe], ['Standard Scaler', scaler])
)

In [ ]:
# Make a pipeline for the model
kmeans_pipe = Pipeline(
    steps=(['Store Pipe', store_pipe], ['KMeans Clustering', kmeans])
)

# Fit the data with the model
kmeans_pipe.fit(store)

Pipeline(steps=[('Store Pipe',
                 Pipeline(steps=[('OneHot Encoder',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False)),
                                 ['Standard Scaler', StandardScaler()]])),
                ['KMeans Clustering', KMeans()]])

In [ ]:
# Make predictions
kmeans_predictions = kmeans_pipe.predict(store)
kmeans_predictions

array([1, 1, 6, ..., 1, 1, 1], dtype=int32)

In [ ]:
# Make a cluster columns
store['Cluster'] = kmeans.labels_

IndexError: ignored

In [ ]:
# Scatter plot of clusters
plt.scatter(store['CompetitionDistance'], store['Promo2'], c=store['Cluster'])
plt.xlabel('Competition Distance')
plt.ylabel('Promo')
plt.title('Store Clustering Based on Competition Distance and Store Promo')

### Light Data Exploration and Data Cleaning for Train Dataframe

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Mynt Project/train.csv')
train

In [ ]:
for column in train.columns:
  print(train[column].value_counts())

The 0 in the StateHoliday column must be cleaned

In [ ]:
# Clean the StateHoliday column
train['StateHoliday'].replace('0',0, inplace=True)

In [ ]:
# Check again for value counts
for column in train.columns:
  print(train[column].value_counts())

In [ ]:
train.info()

There are no null or missing values in the training data

In [ ]:
# Pairplot
sns.pairplot(store)

In [ ]:
sns.pairplot(train)

In [ ]:
# Separate the numerical and categorical variables
store_cat = store[['StoreType', 'Assortment', 'PromoInterval']]
store_num = store.drop(columns=['StoreType', 'Assortment', 'PromoInterval'])

train_cat = train[['Date', 'StateHoliday']]
train_num = train.drop(columns=['Date', 'StateHoliday'])

In [ ]:
# Light exploration for the numerical variables of store dataframe
for column in store_num.columns:
  sns.set_context('talk', font_scale=1.2)
  plt.figure(figsize=(10,4))
  plt.hist(store_num[column])
  plt.xlabel(column)
  plt.ylabel('Count')
  plt.title(f"Distribution of {column}")
  plt.show()

In [ ]:
# Light exploration for the categorical variables of store dataframe
for column in store_cat.columns:
  sns.set_context('talk', font_scale=3)
  plt.figure(figsize=(60,34))
  sns.barplot(store_cat[column].value_counts().index, store_cat[column].value_counts()).set_title(
      f"Distribution of {column}"
  )
  plt.xlabel(column)
  plt.ylabel('Count')
  plt.xticks(rotation=60)
  plt.show()

In [ ]:
# Light exploration of the numerical variables for the train dataframe
for column in train_num.columns:
  sns.set_context('talk', font_scale=1.2)
  plt.figure(figsize=(10,4))
  plt.hist(train_num[column])
  plt.xlabel(column)
  plt.ylabel('Count')
  plt.title(f"Distribution of {column}")
  plt.show()

In [ ]:
# Light exploration for the categorical variables of the train dataframe
for column in train_cat.columns:
  sns.set_context('talk', font_scale=3)
  plt.figure(figsize=(60,34))
  sns.barplot(train_cat[column].value_counts().index, train_cat[column].value_counts()).set_title(
      f"Distribution of {column}"
  )
  plt.xlabel(column)
  plt.ylabel('Count')
  plt.xticks(rotation=60)
  plt.show()